In [1]:
import os
import json
from tqdm.notebook import tqdm
import cv2
import matplotlib.pyplot as plt
import uuid
import subprocess
from multiprocessing import Pool
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import shutil
from timeout_decorator import timeout, TimeoutError

In [2]:
inputFolder = "./input"

In [3]:
def trim(json, likelihood=0.98):
    
    save="./faces/{0}.jpg"
    save_up="./faces_up/{0}.jpg"
    save_middle="./faces_middle/{0}.jpg"
    
    try:
        if json["likelihood"]<likelihood:
            return 
        img = cv2.imread(json["file"])
        #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        x = json["face"]["x"]
        y = json["face"]["y"]
        width = json["face"]["width"]
        height = json["face"]["height"]
        img_up = img[y:y+height, x:x+width,:]
        img_trim = img[y-height//2:y+(height*3)//2, x-width//2:x+(width*3)//2,:]
        img_middle = img[y-height//3:y+(height*4)//3, x-width//3:x+(width*4)//3,:]
        

        cv2.imwrite(save.format(str(uuid.uuid4())), img_trim)
        cv2.imwrite(save_up.format(str(uuid.uuid4())), img_up)
        cv2.imwrite(save_middle.format(str(uuid.uuid4())), img_middle)
    except Exception as e:
        pass
        
@timeout(1200)
def _create_dataset(filename):
    if (filename[-4::] in (".jpg",".png"))==False :
        return
    uid = str(uuid.uuid4())
    tmpFolder="tmp-" + uid
    os.makedirs(tmpFolder)
    shutil.copy(os.path.join(inputFolder, filename), os.path.join(tmpFolder, filename))
    cmd = "ruby /animeface-2009/animeface-ruby/proc_folder.rb {0} {0}/{1}.json".format(tmpFolder, uid)
    subprocess.run(cmd.split(" "))
    with open("{0}/{1}.json".format(tmpFolder, uid), "r") as fp:
        for line in fp:
            data=json.loads(line)
            trim(data)
    
    shutil.rmtree(tmpFolder)

def create_dataset(filename):
    try:
        _create_dataset(filename)
    except TimeoutError:
        print("timeout:",filename)

In [4]:
with Pool() as p:
    imap = p.imap(create_dataset, os.listdir(inputFolder))
    list(tqdm(imap, total=len(os.listdir(inputFolder))))

timeout: f4165ff1-b97d-486f-b1c7-ec551b0c3288.jpg
timeout: d5faea88-5694-4fef-bb58-81808fc726cd.jpg
timeout: 53337a98-344f-4514-ae33-e33b9fee4762.jpg
timeout: 34e22b16-9705-4c4d-bde3-85c78e46196c.jpg
timeout: 1998d18c-a77a-4903-959f-8c4ab1a7f781.jpg
timeout: f945a350-b5e0-43d1-b16d-81e7bb46aab7.jpg
timeout: 3f516d09-ceaf-4e93-9f72-1a2eb9c2c62a.jpg
timeout: ad5f7a0d-93c4-4521-a462-71eb7af07332.jpg
timeout: 9e979890-6465-4091-8be8-658cdc8449a0.jpg
timeout: c80b8239-b5b9-4db9-91d2-19ebffb917d3.jpg
timeout: 4280dc42-b161-42dc-89eb-277d1bde57e6.jpg
timeout: 2de1eb71-cb54-432b-94c0-3fea105ce3c2.jpg
timeout: eb3a3643-027f-41ee-a047-bfd7586aaf3f.jpg
timeout: 4280a362-7890-4d8f-8eed-74f9a1a4e8c7.jpg
timeout: f8e5f269-f787-4dcd-99a5-ce4cc4871285.jpg
timeout: ad6580b9-ca65-4e1e-b865-ef8ddd01eb76.jpg
timeout: a1b3eb39-8a9c-4a71-bd3d-e2051fdc1862.jpg
timeout: 18dcaa3a-2b19-4e57-aa45-c2f01675ecf5.jpg

